#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Create lfi script
* Add uncertain evidence to lfi model as probabilistic clauses

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import os

from src.models import model_schemata as schema
from src.models import build_model as build

In [4]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [5]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/synthetic_data/toy_network_mini/p_model/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/synthetic_data/toy_network_mini/p_model/p_fc.csv')

In [6]:
# replace p0 with P in phosphosite column
data['p_fc']['phosphosite'] = data['p_fc']['phosphosite'].str.replace('p0', 'P')
data['p_fc']

,phosphosite,sample,value,prob,p_dec,p_base,p_inc
0,P,s1,base,0.751050,0.124475,0.751050,0.124475
1,P,s1,dec,0.751050,0.751050,0.247950,0.001000
2,P,s1,inc,0.751050,0.001000,0.247950,0.751050
3,P,s10,base,0.884666,0.057667,0.884666,0.057667
4,P,s10,dec,0.884666,0.884666,0.114334,0.001000
...,...,...,...,...,...,...,...
295,P,s98,dec,0.700061,0.700061,0.298939,0.001000
296,P,s98,inc,0.700061,0.001000,0.298939,0.700061
297,P,s99,base,0.554254,0.222873,0.554254,0.222873
298,P,s99,dec,0.554254,0.554254,0.444746,0.001000


In [7]:
# replace e1 with E in enzyme column 
data['e_ksea']['enzyme'] = data['e_ksea']['enzyme'].str.replace('e1', 'E')
data['e_ksea'] = data['e_ksea'].loc[data['e_ksea']['enzyme'] == 'E']
data['e_ksea']

,enzyme,sample,value,prob,p_dec,p_base,p_inc
0,E,s1,base,0.667479,0.166260,0.667479,0.166260
1,E,s1,dec,0.667479,0.667479,0.331521,0.001000
2,E,s1,inc,0.667479,0.001000,0.331521,0.667479
3,E,s10,base,0.869338,0.065331,0.869338,0.065331
4,E,s10,dec,0.869338,0.869338,0.129662,0.001000
...,...,...,...,...,...,...,...
295,E,s98,dec,0.852893,0.852893,0.146107,0.001000
296,E,s98,inc,0.852893,0.001000,0.146107,0.852893
297,E,s99,base,0.795218,0.102391,0.795218,0.102391
298,E,s99,dec,0.795218,0.795218,0.203782,0.001000


In [8]:
# Mapping data to Problog predicates
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(data['e_ksea'], 'enzyme', 'sample', 'value')
predicates['e_ksea'].add_ad_probabilities(data['e_ksea'], 'p_dec', 'p_base', 'p_inc')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(data['p_fc'], 'phosphosite', 'sample', 'value')
predicates['p_fc'].add_ad_probabilities(data['p_fc'], 'p_dec', 'p_base', 'p_inc')

In [10]:
model_lfi = 'models/synthetic_data/toy_network_mini/p_model/p_model_lfi_o5.pl'
for predicate in predicates: 
    ad_generator = build.ProblogStatementGenerator(predicates[predicate])
    clauses = ad_generator.generate_ad_clauses()
    build.insert_statements(model=model_lfi, statements=clauses, location='%% p1::{}'.format(predicate)) # insert into Problog file